Part 2: Practical Implementation
Task 1: Edge AI Prototype

Tools: TensorFlow Lite, Raspberry Pi/Colab (simulation).

Goal:


*  Train a lightweight image classification model (e.g., recognizing recyclable
items).
*   Convert the model to TensorFlow Lite and test it on a sample dataset.


*   Explain how Edge AI benefits real-time applications.



Deliverable: Code + report with accuracy metrics and deployment steps.

In [1]:
import tensorflow as tf
import numpy as np
import os
import pathlib

# ==========================================
# CONFIGURATION
# ==========================================
IMG_SIZE = 32  # Small size for speed/Edge AI demo
BATCH_SIZE = 32
EPOCHS = 5     # Low epoch count for demonstration purposes
MODEL_PATH = 'recycling_model.tflite'

print(f"TensorFlow Version: {tf.__version__}")

# ==========================================
# 1. DATASET PREPARATION
# ==========================================
# NOTE: In a real-world scenario, you would load a custom dataset from a directory
# using tf.keras.utils.image_dataset_from_directory.
# For this assignment demo, we use CIFAR-10 as a proxy to ensure the code is runnable.
# We will map:
# Vehicles (0, 1, 8, 9) -> Class 0: "Recyclable (Metal/Plastic)"
# Animals (2, 3, 4, 5, 6, 7) -> Class 1: "Organic/Trash"

def load_and_preprocess_data():
    print("Loading proxy dataset (CIFAR-10)...")
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

    # Normalize pixel values to be between 0 and 1
    x_train, x_test = x_train / 255.0, x_test / 255.0

    # Remap labels for our "Recycling" theme
    # Original: 0:air, 1:auto, 2:bird, 3:cat, 4:deer, 5:dog, 6:frog, 7:horse, 8:ship, 9:truck
    # New: 0 (Recyclable), 1 (Organic)
    recyclable_indices = [0, 1, 8, 9]

    def remap_labels(y):
        new_y = np.zeros_like(y)
        for i, label in enumerate(y):
            if label[0] in recyclable_indices:
                new_y[i] = 0 # Recyclable
            else:
                new_y[i] = 1 # Organic
        return new_y

    y_train_bin = remap_labels(y_train)
    y_test_bin = remap_labels(y_test)

    print("Dataset loaded and remapped for binary classification.")
    return (x_train, y_train_bin), (x_test, y_test_bin)

(x_train, y_train), (x_test, y_test) = load_and_preprocess_data()

# ==========================================
# 2. MODEL ARCHITECTURE (MobileNetV2)
# ==========================================
# We use MobileNetV2 as it is optimized for mobile and edge devices.
print("Building MobileNetV2 model...")

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet',
    alpha=0.35 # Reduced width multiplier for an even smaller model
)
base_model.trainable = False # Freeze base model

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid') # Binary classification
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

# ==========================================
# 3. TRAINING
# ==========================================
print("Starting training...")
history = model.fit(x_train, y_train,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    batch_size=BATCH_SIZE)

# ==========================================
# 4. CONVERSION TO TENSORFLOW LITE
# ==========================================
print("\nConverting model to TensorFlow Lite...")

# Initialize converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# OPTIMIZATION: QUANTIZATION
# This reduces model size and latency with minimal accuracy loss
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Convert the model
tflite_model = converter.convert()

# Save the model
with open(MODEL_PATH, 'wb') as f:
    f.write(tflite_model)

print(f"Model saved to {MODEL_PATH}")

# Compare sizes
keras_model_size = os.path.getsize('recycling_model.keras') if os.path.exists('recycling_model.keras') else 0
tflite_model_size = os.path.getsize(MODEL_PATH)
print(f"TFLite Model Size: {tflite_model_size / 1024:.2f} KB")

# ==========================================
# 5. TESTING THE TFLITE MODEL
# ==========================================
print("\nRunning inference on TFLite model...")

# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
interpreter.allocate_tensors()

# Get input and output details.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test on a single image from the test set
input_shape = input_details[0]['shape']
input_data = np.expand_dims(x_test[0], axis=0).astype(np.float32)

# Set the tensor to point to the input data to be inferred
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run the inference
interpreter.invoke()

# Retrieve the result
output_data = interpreter.get_tensor(output_details[0]['index'])
prediction = "Organic/Trash" if output_data[0][0] > 0.5 else "Recyclable"
actual = "Organic/Trash" if y_test[0][0] > 0.5 else "Recyclable"

print(f"Test Image Label: {actual}")
print(f"TFLite Prediction: {prediction}")
print(f"Raw Output Probability: {output_data[0][0]:.4f}")

print("\n=== EDGE AI PROTOTYPE COMPLETE ===")

TensorFlow Version: 2.19.0
Loading proxy dataset (CIFAR-10)...
170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step
Dataset loaded and remapped for binary classification.
Building MobileNetV2 model...


/tmp/ipython-input-3657573514.py:61: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


2019640/2019640 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_0.35_224            │ (None, 1, 1, 1280)     │       410,208 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 411,489 (1.57 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 410,208 (1.56 MB)

Starting training...
Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 22s 12ms/step - accuracy: 0.7277 - loss: 0.5898 - val_accuracy: 0.7868 - val_loss: 0.4799
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7782 - loss: 0.4858 - val_accuracy: 0.7929 - val_loss: 0.4586
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.7766 - loss: 0.4774 - val_accuracy: 0.7946 - val_loss: 0.4516
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.7776 - loss: 0.4753 - val_accuracy: 0.7960 - val_loss: 0.4489
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 11ms/step - accuracy: 0.7793 - loss: 0.4726 - val_accuracy: 0.7971 - val_loss: 0.4471

Converting model to TensorFlow Lite...
Saved artifact at '/tmp/tmp2davf896'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures

/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Task 2: AI-Driven IoT Concept

Scenario: Design a smart agriculture simulation system using AI and IoT.

Requirements:



*  List sensors needed (e.g., soil moisture, temperature).
*   Sketch a data flow diagram (AI processing sensor data).


*   Sketch a data flow diagram (AI processing sensor data).





Tips: Be free to use any tools of your choice

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# ==========================================
# 1. IOT DATA SIMULATION
# ==========================================
# We simulate data for a crop cycle (e.g., Wheat or Corn)
# Features: Moisture(%), Temperature(C), Nitrogen(ppm), Phosphorus(ppm), Potassium(ppm), pH
print("Generating synthetic IoT sensor data...")

np.random.seed(42)
n_samples = 1000

# Generate random environmental data within realistic ranges
data = {
    'soil_moisture': np.random.uniform(10, 90, n_samples),      # %
    'temperature': np.random.uniform(15, 35, n_samples),        # Celsius
    'nitrogen': np.random.uniform(0, 100, n_samples),           # ppm
    'phosphorus': np.random.uniform(0, 100, n_samples),         # ppm
    'potassium': np.random.uniform(0, 100, n_samples),          # ppm
    'ph_level': np.random.uniform(5.0, 8.5, n_samples)          # pH scale
}

df = pd.DataFrame(data)

# Define a synthetic "Yield" formula to simulate biological reality
# Yield is high if:
# - Moisture is between 40% and 70%
# - Temp is between 20C and 30C
# - Nutrients (NPK) are high
# - pH is neutral (6.0 - 7.5)
def calculate_synthetic_yield(row):
    yield_score = 1000 # Base yield (kg)

    # Moisture penalty
    if row['soil_moisture'] < 30: yield_score -= 300
    if row['soil_moisture'] > 80: yield_score -= 200 # Waterlogging

    # Temp penalty
    if row['temperature'] > 32 or row['temperature'] < 18: yield_score -= 150

    # Nutrient bonus
    yield_score += (row['nitrogen'] + row['phosphorus'] + row['potassium']) * 2

    # pH penalty (optimal 6-7)
    dist_from_optimal = abs(row['ph_level'] - 6.5)
    yield_score -= dist_from_optimal * 100

    # Add random noise
    yield_score += np.random.normal(0, 50)

    return max(0, yield_score) # Yield cannot be negative

df['crop_yield'] = df.apply(calculate_synthetic_yield, axis=1)

print("Data generation complete. Sample:")
print(df.head(3))

# ==========================================
# 2. DATA PREPROCESSING
# ==========================================
X = df.drop('crop_yield', axis=1)
y = df['crop_yield']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# 3. AI MODEL TRAINING (Random Forest)
# ==========================================
print("\nTraining Random Forest Regressor...")
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"Mean Absolute Error: {mae:.2f} kg")
print(f"R^2 Score: {r2:.4f} (1.0 is perfect)")

# ==========================================
# 4. SIMULATION: REAL-TIME PREDICTION
# ==========================================
print("\n--- Real-Time Sensor Simulation ---")

# Simulating a "Live" reading from the IoT Gateway
current_sensor_reading = {
    'soil_moisture': 25.5,  # Dry!
    'temperature': 28.0,    # Good
    'nitrogen': 40.0,       # Moderate
    'phosphorus': 45.0,     # Moderate
    'potassium': 42.0,      # Moderate
    'ph_level': 6.2         # Good
}

input_df = pd.DataFrame([current_sensor_reading])
predicted_yield = model.predict(input_df)[0]

print(f"Live Sensor Readings: {current_sensor_reading}")
print(f"AI Predicted Yield: {predicted_yield:.2f} kg")

# Rule-based Logic based on prediction and inputs
if predicted_yield < 800:
    print("ALERT: Low yield predicted!")
    if current_sensor_reading['soil_moisture'] < 30:
        print("ACTION: Triggering Automated Irrigation System (Moisture Critical)")
    elif current_sensor_reading['nitrogen'] < 30:
        print("ACTION: Recommend Fertilizer Application")
else:
    print("STATUS: Optimal conditions maintained.")

print("\n=== SMART AGRICULTURE SIMULATION COMPLETE ===")

Generating synthetic IoT sensor data...
Data generation complete. Sample:
   soil_moisture  temperature   nitrogen  phosphorus  potassium  ph_level  \
0      39.963210    18.702659  26.170568   67.270299  57.199588  6.377724   
1      86.057145    25.838019  24.697880   79.668140  80.543233  6.657025   
2      68.559515    32.458917  90.625458   25.046790  76.016093  7.990916   

    crop_yield  
0  1236.747289  
1  1177.786395  
2  1098.832462  

Training Random Forest Regressor...
Model Performance:
Mean Absolute Error: 62.48 kg
R^2 Score: 0.8451 (1.0 is perfect)

--- Real-Time Sensor Simulation ---
Live Sensor Readings: {'soil_moisture': 25.5, 'temperature': 28.0, 'nitrogen': 40.0, 'phosphorus': 45.0, 'potassium': 42.0, 'ph_level': 6.2}
AI Predicted Yield: 920.83 kg
STATUS: Optimal conditions maintained.

=== SMART AGRICULTURE SIMULATION COMPLETE ===
